In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'Breast Cancer Prediction (From scratch).ipynb',
 'data.csv',
 'model.png']

## Reading the data, and cleaning up missing values

In [3]:
col_names = ["id", "clump_thickness", "un_cell_size", "un_cell_shape", "marginal_adheshion", "single_eph_cell_size", "bare_nuclei", "bland_chromatin", "normal_nucleoli", "mitoses", "class"]
data = pd.read_csv('./data.csv', names = col_names, header = None)

In [4]:
data.head()

,id,clump_thickness,un_cell_size,un_cell_shape,marginal_adheshion,single_eph_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
data.drop('id', inplace = True, axis = 1)

In [6]:
data.head()

,clump_thickness,un_cell_size,un_cell_shape,marginal_adheshion,single_eph_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   clump_thickness       699 non-null    int64 
 1   un_cell_size          699 non-null    int64 
 2   un_cell_shape         699 non-null    int64 
 3   marginal_adheshion    699 non-null    int64 
 4   single_eph_cell_size  699 non-null    int64 
 5   bare_nuclei           699 non-null    object
 6   bland_chromatin       699 non-null    int64 
 7   normal_nucleoli       699 non-null    int64 
 8   mitoses               699 non-null    int64 
 9   class                 699 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


In [8]:
data = data[data["bare_nuclei"] != '?' ]
data.bare_nuclei = pd.to_numeric(data.bare_nuclei)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   clump_thickness       683 non-null    int64
 1   un_cell_size          683 non-null    int64
 2   un_cell_shape         683 non-null    int64
 3   marginal_adheshion    683 non-null    int64
 4   single_eph_cell_size  683 non-null    int64
 5   bare_nuclei           683 non-null    int64
 6   bland_chromatin       683 non-null    int64
 7   normal_nucleoli       683 non-null    int64
 8   mitoses               683 non-null    int64
 9   class                 683 non-null    int64
dtypes: int64(10)
memory usage: 58.7 KB


## Data preprocessing

In [30]:
data = data.sample(frac = 1)

In [31]:
X = data.drop('class', axis = 1)
y = data['class']

In [32]:
data_stats = X.describe().transpose()

In [33]:
def norm(df):
    return (df - data_stats['mean']) / data_stats['std']

In [34]:
X = norm(X)

In [35]:
X.describe()

,clump_thickness,un_cell_size,un_cell_shape,marginal_adheshion,single_eph_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses
count,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02,6.830000e+02
mean,1.105346e-16,1.560489e-17,-7.802446e-17,5.981875e-17,-1.560489e-17,-4.551427e-18,4.161304e-17,5.136610e-17,7.542364e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.220297e+00,-7.016978e-01,-7.412304e-01,-6.388973e-01,-1.005027e+00,-6.983413e-01,-9.981216e-01,-6.124785e-01,-3.481446e-01
25%,-8.657829e-01,-7.016978e-01,-7.412304e-01,-6.388973e-01,-5.552016e-01,-6.983413e-01,-5.899078e-01,-6.124785e-01,-3.481446e-01
50%,-1.567545e-01,-7.016978e-01,-7.412304e-01,-6.388973e-01,-5.552016e-01,-6.983413e-01,-1.816940e-01,-6.124785e-01,-3.481446e-01
75%,5.522740e-01,6.032977e-01,5.971975e-01,4.083832e-01,3.444489e-01,6.738310e-01,6.347336e-01,3.702689e-01,-3.481446e-01
max,1.970331e+00,2.234542e+00,2.270232e+00,2.502944e+00,3.043400e+00,1.771569e+00,2.675803e+00,2.335764e+00,4.846139e+00


In [36]:
y.unique()

array([2, 4], dtype=int64)

In [37]:
y = np.where(y == 2, 0, 1)

In [38]:
y

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [39]:
X_train, X_test, y_train, y_test = X[:int(len(data)*0.8)], X[int(len(data)*0.8):], y[:int(len(data)*0.8)], y[int(len(data)*0.8):]

In [40]:
X_train.shape

(546, 9)

In [41]:
y_train.shape

(546,)

## Building a model from scratch

The model will be of the following architecture:
* Input Layer
* Dense(128) with relu activation
* Dropout of 0.3
* Dense(1) with sigmoid activation

In [42]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense, Dropout, Input
from tensorflow.keras.utils import plot_model

In [43]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc_1 = Dense(128, activation = 'relu')
        self.output_ = Dense(1, activation = 'sigmoid')
        self.dropout = Dropout(0.3)
    
    def call(self, inputs):
        x = self.fc_1(inputs)
        x = self.dropout(x)

        return self.output_(x)

In [44]:
input_layer = Input(shape = (9))
output_layer = MyModel()(input_layer)

model = Model(inputs = input_layer, outputs = output_layer)

In [45]:
model.compile(metrics = ['accuracy'],
             optimizer = 'adam',
             loss = tf.keras.losses.BinaryCrossentropy())

In [46]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
18/18 [==============================] - 1s 12ms/step - loss: 0.5907 - accuracy: 0.7420 - val_loss: 0.3693 - val_accuracy: 0.9562
Epoch 2/10
18/18 [==============================] - 0s 6ms/step - loss: 0.2958 - accuracy: 0.9703 - val_loss: 0.2288 - val_accuracy: 0.9562
Epoch 3/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1779 - accuracy: 0.9748 - val_loss: 0.1686 - val_accuracy: 0.9562
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1359 - accuracy: 0.9678 - val_loss: 0.1408 - val_accuracy: 0.9562
Epoch 5/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1004 - accuracy: 0.9770 - val_loss: 0.1258 - val_accuracy: 0.9562
Epoch 6/10
18/18 [==============================] - 0s 7ms/step - loss: 0.0938 - accuracy: 0.9687 - val_loss: 0.1172 - val_accuracy: 0.9562
Epoch 7/10
18/18 [==============================] - 0s 7ms/step - loss: 0.0977 - accuracy: 0.9615 - val_loss: 0.1123 - val_accuracy: 0.9562
Epoch 8/10
18/18 [=

## Building the Layers from scratch

Here we will be building a Dense Layer / fully connected layer and a Dropout Layer from scratch

In [47]:
class MyDense(Layer):
    def __init__(self, units, activation = None):
        super(MyDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
    
    def build(self, input_shape):
        
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            name = 'kernel',
            initial_value = w_init(shape = (input_shape[-1], self.units), dtype = 'float32'),
            trainable = True
        )
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            name = 'bias',
            initial_value = b_init(shape = (self.units,), dtype = 'float32'),
            trainable = True
        )
        super().build(input_shape)
        
    def call(self, inputs):
        return self.activation(tf.add(tf.matmul(inputs, self.w), self.b))

In [99]:
class MyDropout(Layer):
    def __init__(self, drop_pct = 0.0):
        super(MyDropout, self).__init__()
        self.drop_pct = drop_pct
        
    def build(self, input_shape):
        zeros_tensor = tf.zeros(shape = (int(input_shape[-1] * self.drop_pct), ))
        ones_tensor = tf.ones(shape = (input_shape[-1] - int(input_shape[-1] * self.drop_pct), ))
        self.drop_matrix = tf.Variable(
            name = 'Dropout Matrix',
            initial_value = tf.concat([zeros_tensor, ones_tensor], 0),
            trainable = False)
        
        super().build(input_shape)
    
    def call(self, inputs):
        return inputs * tf.random.shuffle(self.drop_matrix)

In [100]:
input_layer = Input(shape=(9))
x = MyDense(128, activation = 'relu')(input_layer)
x = MyDropout(0.4)(x)
x = MyDense(128, activation = 'relu')(x)
output_layer = MyDense(1, activation = 'sigmoid')(x)

In [101]:
model = Model(inputs = [input_layer], outputs = output_layer)

In [102]:
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 9)]               0         
_________________________________________________________________
my_dense_21 (MyDense)        (None, 128)               1280      
_________________________________________________________________
my_dropout_8 (MyDropout)     (None, 128)               128       
_________________________________________________________________
my_dense_22 (MyDense)        (None, 128)               16512     
_________________________________________________________________
my_dense_23 (MyDense)        (None, 1)                 129       
Total params: 18,049
Trainable params: 17,921
Non-trainable params: 128
_________________________________________________________________


In [103]:
model.compile(
    metrics = ['accuracy'],
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam())

In [104]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
18/18 [==============================] - 1s 19ms/step - loss: 0.6694 - accuracy: 0.8931 - val_loss: 0.5749 - val_accuracy: 0.9562
Epoch 2/10
18/18 [==============================] - 0s 7ms/step - loss: 0.5055 - accuracy: 0.9738 - val_loss: 0.3040 - val_accuracy: 0.9562
Epoch 3/10
18/18 [==============================] - 0s 7ms/step - loss: 0.2129 - accuracy: 0.9765 - val_loss: 0.1338 - val_accuracy: 0.9562
Epoch 4/10
18/18 [==============================] - 0s 7ms/step - loss: 0.1115 - accuracy: 0.9679 - val_loss: 0.1026 - val_accuracy: 0.9562
Epoch 5/10
18/18 [==============================] - 0s 8ms/step - loss: 0.0826 - accuracy: 0.9742 - val_loss: 0.1016 - val_accuracy: 0.9562
Epoch 6/10
18/18 [==============================] - 0s 8ms/step - loss: 0.0617 - accuracy: 0.9812 - val_loss: 0.1000 - val_accuracy: 0.9562
Epoch 7/10
18/18 [==============================] - 0s 8ms/step - loss: 0.0641 - accuracy: 0.9772 - val_loss: 0.1070 - val_accuracy: 0.9562
Epoch 8/10
18/18 [=

In [128]:
import matplotlib.pyplot as plt

In [129]:
data.columns

Index(['clump_thickness', 'un_cell_size', 'un_cell_shape',
       'marginal_adheshion', 'single_eph_cell_size', 'bare_nuclei',
       'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class'],
      dtype='object')